In [1]:
import cv2
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


folder = "datas/"
gestures = {1: "Stop sign", 2: "Thumbs down", 3: "Waving", 4: "Pointing",
            5: "Calling someone", 6: "Thumbs up", 7: "Wave someone away",9: "Others"}#, 0: "Nodding",  8: "Shaking head"}
gestures_len = 8
sequence_len = 10
size = (64,64)

def rolling_window2D(a,n,step=3):
    # a: 2D Input array 
    # n: Group/sliding window length
    return a[np.arange(a.shape[0]-n+1)[:,None] + np.arange(n)][::step, :]

def load_csv(Xd, Yd, sequence_len = 30):
    X=[]
    Y=[]
    for x1, y1 in zip(Xd, Yd):
        try:
            frames = []
            cap = cv2.VideoCapture(x1)
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                  cap.release()
                  cv2.destroyAllWindows()
                  break
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frame = cv2.resize(frame, size) 
                frame = frame/255
                frames.append(frame)
            frames = np.array(frames)
            x = rolling_window2D(frames,sequence_len)
            X.extend(list(x))
            y = np.ones(len(x),dtype=int)*y1
            #y = np.ones(len(x),dtype=int)*i
            Y.extend(list(y))
        except Exception as e: 
            print(e)
            pass # Exception as e
    return np.array(X),np.array(Y)

Xd=[]
Yd=[]
folder_list = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]
for folder_i in folder_list:
    for i, k in enumerate(list(gestures.keys())):
        dir_list = os.listdir(folder+folder_i+"/gesture/"+str(k))
        for d in dir_list:
            if d!="desktop.ini":
                Xd.append(folder+folder_i+"/gesture/"+str(k)+"/"+d)   
                Yd.append(i)
X_train, X_test, y_train, y_test = train_test_split(Xd, Yd, test_size=0.2, random_state=42, stratify=Yd)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

X_train,y_train = load_csv(X_train,y_train,sequence_len)
X_test,y_test = load_csv(X_test,y_test,sequence_len)
X_val,y_val = load_csv(X_val,y_val,sequence_len)

y_train = to_categorical(y_train).astype(int)
y_val = to_categorical(y_val).astype(int)
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

2023-06-20 10:51:58.799312: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-20 10:51:59.090743: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zany/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2023-06-20 10:51:59.090774: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-20 10:51:59.134302: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to reg

(3880, 10, 64, 64) (3880, 8)
(970, 10, 64, 64) (970, 8)
(1228, 10, 64, 64) (1228,)


In [2]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing import image
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

def build_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(10, 64, 64)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(len(gestures), activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print("model loaded")

model loaded


2023-06-20 10:52:38.388685: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-20 10:52:38.388929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zany/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2023-06-20 10:52:38.389009: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/zany/.local/lib/python3.8/site-packages/cv2/../../lib64:/opt/ros/noetic/lib:/opt/ros/noetic/lib/x86_64-linux-gnu
2023-06-20 10:52:38.389181: W tensorflow/stream_ex

In [3]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=16, epochs=20)

Epoch 1/20


2023-06-20 10:52:38.764016: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 635699200 exceeds 10% of free system memory.


243/243 [==============================] - 3s 12ms/step - loss: 0.9395 - accuracy: 0.6737 - val_loss: 8.8970 - val_accuracy: 0.1052
Epoch 2/20
243/243 [==============================] - 3s 12ms/step - loss: 0.2140 - accuracy: 0.9340 - val_loss: 11.3713 - val_accuracy: 0.1000
Epoch 3/20
243/243 [==============================] - 3s 12ms/step - loss: 0.1256 - accuracy: 0.9601 - val_loss: 13.1264 - val_accuracy: 0.1000
Epoch 4/20
243/243 [==============================] - 3s 12ms/step - loss: 0.0666 - accuracy: 0.9786 - val_loss: 12.8293 - val_accuracy: 0.1072
Epoch 5/20
243/243 [==============================] - 3s 11ms/step - loss: 0.0507 - accuracy: 0.9840 - val_loss: 15.4264 - val_accuracy: 0.1000
Epoch 6/20
243/243 [==============================] - 3s 12ms/step - loss: 0.0425 - accuracy: 0.9866 - val_loss: 14.7591 - val_accuracy: 0.1000
Epoch 7/20
243/243 [==============================] - 3s 12ms/step - loss: 0.0177 - accuracy: 0.9948 - val_loss: 16.3257 - val_accuracy: 0.1000
Epoc

In [4]:
# Evaluate the model
y_test_new = to_categorical(y_test,8)
print(y_test_new.shape)
loss, accuracy = model.evaluate(X_test, y_test_new)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

(1228, 8)
39/39 [==============================] - 0s 5ms/step - loss: 13.4380 - accuracy: 0.1808
Test Loss: 13.438045501708984
Test Accuracy: 0.180781751871109
